In [1]:
import pandas as pd 
import numpy
import matplotlib.pyplot as plt 
from pyspark.sql import SparkSession, Row

In [2]:
spark = SparkSession.builder.appName("BaPoints").getOrCreate()

In [108]:
points = spark.read.json('../Datasets/ba_points/points_50000_new.json')
points = points.drop('_id').selectExpr('_vehicle._trip._route_id as route_id',
                              '_vehicle._trip._trip_id as trip_id',
                              '_vehicle._stop_id as last_stop_id',
                              '_vehicle._timestamp as timestamp',
                              '_vehicle._vehicle._id as vehicle_id',
                              '_vehicle._vehicle._label as vehicle_label',
                              '_vehicle._trip._start_time as start_time',
                              '_vehicle._trip._start_date as start_date',         
                              '_vehicle._position._latitude as latitude',
                              '_vehicle._position._longitude as longitude',
                              '_vehicle._position._speed as speed')
points

DataFrame[route_id: string, trip_id: string, last_stop_id: string, timestamp: bigint, vehicle_id: string, vehicle_label: string, start_time: string, start_date: string, latitude: double, longitude: double, speed: double]

In [1]:
routes = spark.read.csv('../Datasets/ba_points/routes.txt', inferSchema=True, header=True)
routes = routes.drop('route_url','route_color', 'route_text_color')
routes

NameError: name 'spark' is not defined

In [59]:
stop_trips = spark.read.csv('../Datasets/ba_points/stop_times.txt', inferSchema=True, header=True)
stop_trips = stop_trips.drop('arrival_time', 'departure_time', 'stop_headsign',
                             'pickup_type', 'drop_off_type', 'shape_dist_traveled')
stop_trips.show()

+-------+-------+-------------+
|trip_id|stop_id|stop_sequence|
+-------+-------+-------------+
| 101HI0|2033214|           15|
| 101HI0| 202889|           16|
| 101HI0|2033139|           17|
| 101HI0| 201738|           18|
| 101HI0| 201633|           19|
| 101HI0| 201525|           20|
| 101HI0| 202096|           21|
| 101HI0| 202089|           22|
| 101HI0| 205634|           23|
| 101HI0| 202051|           24|
| 101HI0|2032198|           25|
| 101HI0| 201476|           26|
| 101HI0| 206126|           27|
| 101HI0| 201398|           28|
| 101HI0| 205686|           29|
| 101HI0| 205659|           30|
| 101HI0|2033223|           31|
| 101HI0|  20796|           32|
| 101HI0| 203054|           33|
| 101HI0| 201079|           34|
+-------+-------+-------------+
only showing top 20 rows



In [71]:
trips = spark.read.csv('../Datasets/ba_points/trips.txt', inferSchema=True, header=True)
trips = trips.drop('block_id', 'shape_id', 'wheelchair_accessible', 'bikes_allowed')
trips.where(trips.trip_id == '229003-1').show()

+--------+----------+--------+-------------+---------------+------------+-----------+
|route_id|service_id| trip_id|trip_headsign|trip_short_name|direction_id|exceptional|
+--------+----------+--------+-------------+---------------+------------+-----------+
|    3713|         2|229003-1|   a Bº Nuevo|     3713SI0057|           0|          0|
+--------+----------+--------+-------------+---------------+------------+-----------+



In [66]:
stops = spark.read.csv('../Datasets/ba_points/stops.txt', inferSchema=True, header=True)
stops.drop('stop_timezone', 'wheelchair_boarding', 'stop_desc', 'zone_id', 'stop_url', 
           'location_type', 'parent_station').where(stops.stop_id == '6539109469').show()

+----------+---------+--------------------+----------+---------+
|   stop_id|stop_code|           stop_name|  stop_lat| stop_lon|
+----------+---------+--------------------+----------+---------+
|6539109469|     null|AVENIDA PATRICIOS...|-34.684054|-58.68259|
+----------+---------+--------------------+----------+---------+



In [42]:
trips_with_stops = trips.join(stop_trips, 'trip_id').orderBy('route_id', 'stop_sequence')
trips_with_stops

DataFrame[trip_id: string, route_id: int, service_id: string, trip_headsign: string, trip_short_name: string, direction_id: int, stop_id: bigint, stop_sequence: int]